In [79]:
import pandas as pd
import re
import torch
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from torch.nn import functional as F
from transformers import DataCollatorWithPadding, Trainer
from sklearn.metrics import accuracy_score, classification_report
from pydantic import BaseModel
import uvicorn
import threading
import requests
from fastapi import FastAPI, HTTPException, Request
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import logging
from datetime import datetime
import json
from typing import Dict, Any
from fastapi.responses import JSONResponse
import time
from transformers import pipeline

In [9]:
train_file = 'C:/Users/hafiz/Documents/Gomycode/Bitext_Sample_Customer_Service_Training_Dataset.csv'

In [10]:
df_train = pd.read_csv(train_file)

In [11]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6539 entries, 0 to 6538
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   utterance  6539 non-null   object
 1   intent     6539 non-null   object
 2   category   6539 non-null   object
 3   tags       6539 non-null   object
dtypes: object(4)
memory usage: 204.5+ KB


In [12]:
df_train.head()

,utterance,intent,category,tags
0,would it be possible to cancel the order I made?,cancel_order,ORDER,BIP
1,cancelling order,cancel_order,ORDER,BK
2,I need assistance canceling the last order I h...,cancel_order,ORDER,B
3,problem with canceling the order I made,cancel_order,ORDER,B
4,I don't know how to cancel the order I made,cancel_order,ORDER,B


In [13]:
df_train = df_train[['utterance', 'intent']]

In [14]:
df_train = df_train.drop_duplicates().dropna()

In [15]:
def clean_text(text):
    text = text.lower()  
    text = re.sub(r"[^\w\s]", "", text) 
    text = text.strip()  
    return text

In [16]:
df_train["utterance"] = df_train["utterance"].apply(clean_text)

In [17]:
df_train.head()

,utterance,intent
0,would it be possible to cancel the order i made,cancel_order
1,cancelling order,cancel_order
2,i need assistance canceling the last order i h...,cancel_order
3,problem with canceling the order i made,cancel_order
4,i dont know how to cancel the order i made,cancel_order


In [18]:
train_file = 'C:/Users/hafiz/Documents/Gomycode/Bitext_Sample_Customer_Service_Training_Dataset.csv'
test_file = 'C:/Users/hafiz/Documents/Gomycode/Bitext_Sample_Customer_Service_Testing_Dataset.csv'

In [19]:
df_train = pd.read_csv(train_file)[['utterance', 'intent']]
df_test = pd.read_csv(test_file)[['utterance', 'intent']]

In [20]:
df_train = df_train.drop_duplicates().dropna()
df_test = df_test.drop_duplicates().dropna()

In [21]:
def clean_text(text):
    return text.lower().strip()

In [22]:
df_train["utterance"] = df_train["utterance"].apply(clean_text)
df_test["utterance"] = df_test["utterance"].apply(clean_text)

In [23]:
intent_labels = {label: idx for idx, label in enumerate(df_train["intent"].unique())}
df_train["intent"] = df_train["intent"].map(intent_labels)
df_test["intent"] = df_test["intent"].map(intent_labels)

In [24]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [25]:
train_dataset = Dataset.from_pandas(df_train.rename(columns={"intent": "labels"}))
test_dataset = Dataset.from_pandas(df_test.rename(columns={"intent": "labels"}))

In [26]:
def tokenize_data(examples):
    return tokenizer(examples["utterance"], padding="max_length", truncation=True)

In [27]:
train_dataset = train_dataset.map(tokenize_data, batched=True, remove_columns=["utterance"])
test_dataset = test_dataset.map(tokenize_data, batched=True, remove_columns=["utterance"])

Map:   0%|          | 0/6539 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [28]:
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [29]:
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", 
    num_labels=len(intent_labels)
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = F.cross_entropy(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [31]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [32]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
)

C:\Users\hafiz\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [33]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)

In [34]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.228600,0.131169
2,0.030900,0.020303
3,0.019700,0.013985


TrainOutput(global_step=1227, training_loss=0.4875375621306187, metrics={'train_runtime': 21402.9304, 'train_samples_per_second': 0.917, 'train_steps_per_second': 0.057, 'total_flos': 2599771523761152.0, 'train_loss': 0.4875375621306187, 'epoch': 3.0})

In [35]:
model.save_pretrained("chatbot_model")
tokenizer.save_pretrained("chatbot_model")

('chatbot_model\\tokenizer_config.json',
 'chatbot_model\\special_tokens_map.json',
 'chatbot_model\\vocab.txt',
 'chatbot_model\\added_tokens.json')

In [38]:
print("Training complete! Model saved in 'chatbot_model'")

Training complete! Model saved in 'chatbot_model'


In [43]:
def evaluate_model(model, dataset, intent_labels):
    model.eval()
    all_preds = []
    all_labels = []
    
    id_to_label = {v: k for k, v in intent_labels.items()}
    
    for sample in dataset:
        inputs = {
            "input_ids": sample["input_ids"].unsqueeze(0),
            "attention_mask": sample["attention_mask"].unsqueeze(0)
        }
        label = sample["labels"].item()
        
        with torch.no_grad():
            outputs = model(**inputs)
        
        pred = torch.argmax(outputs.logits, dim=1).item()
        all_preds.append(pred)
        all_labels.append(label)
    
    pred_labels = [id_to_label[p] for p in all_preds]
    true_labels = [id_to_label[l] for l in all_labels]
    
    accuracy = accuracy_score(true_labels, pred_labels)
    report = classification_report(true_labels, pred_labels)
    
    return accuracy, report

In [44]:
eval_model = DistilBertForSequenceClassification.from_pretrained("chatbot_model")
eval_tokenizer = DistilBertTokenizer.from_pretrained("chatbot_model")

In [45]:
eval_df = pd.read_csv(test_file)[['utterance', 'intent']].drop_duplicates().dropna()
eval_df["utterance"] = eval_df["utterance"].apply(clean_text)
eval_dataset = Dataset.from_pandas(eval_df)

In [46]:
def eval_tokenize(examples):
    return eval_tokenizer(examples["utterance"], padding="max_length", truncation=True)

eval_dataset = eval_dataset.map(eval_tokenize, batched=True)

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [47]:
eval_dataset = eval_dataset.map(lambda x: {"labels": intent_labels[x["intent"]]})

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [48]:
eval_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [49]:
accuracy, report = evaluate_model(eval_model, eval_dataset, intent_labels)

In [50]:
print(f"\nModel Evaluation Results:")
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(report)


Model Evaluation Results:
Accuracy: 1.0000
Classification Report:
                          precision    recall  f1-score   support

            cancel_order       1.00      1.00      1.00        25
            change_order       1.00      1.00      1.00        39
 change_shipping_address       1.00      1.00      1.00        41
  check_cancellation_fee       1.00      1.00      1.00        26
           check_invoice       1.00      1.00      1.00        31
   check_payment_methods       1.00      1.00      1.00        26
     check_refund_policy       1.00      1.00      1.00        37
               complaint       1.00      1.00      1.00        23
contact_customer_service       1.00      1.00      1.00        24
     contact_human_agent       1.00      1.00      1.00        42
          create_account       1.00      1.00      1.00        25
          delete_account       1.00      1.00      1.00        29
        delivery_options       1.00      1.00      1.00        32
        

In [78]:
app = FastAPI()

class Query(BaseModel):
    text: str

@app.post("/predict")
async def predict_intent(query: Query):
    try:
        text = clean_text(query.text)
        inputs = eval_tokenizer(text, return_tensors="pt", padding="max_length", truncation=True)
        
        with torch.no_grad():
            outputs = eval_model(**inputs)
        
        pred_id = torch.argmax(outputs.logits, dim=1).item()
        intent = list(intent_labels.keys())[list(intent_labels.values()).index(pred_id)]
        
        return {
            "text": query.text,
            "intent": intent,
            "confidence": torch.softmax(outputs.logits, dim=1)[0][pred_id].item()
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/intents")
async def get_intents():
    return {"intents": list(intent_labels.keys())}

In [58]:
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=run_server, daemon=True)
thread.start()

print("FastAPI server is running on http://localhost:8000")

FastAPI server is running on http://localhost:8000


INFO:     Started server process [4024]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:59100 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:59100 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:59103 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:59107 - "GET /intents HTTP/1.1" 200 OK
INFO:     127.0.0.1:59140 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:59140 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:59168 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:59170 - "GET /intents HTTP/1.1" 200 OK
INFO:     127.0.0.1:59253 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:59255 - "GET /intents HTTP/1.1" 200 OK


In [81]:
import json

with open("label_mapping.json", "w") as f:
    json.dump(intent_labels, f)

In [61]:
response = requests.post("http://localhost:8000/predict", json={"text": "How do I reset my password?"})
print(response.json())

{'text': 'How do I reset my password?', 'intent': 'recover_password', 'confidence': 0.9861235618591309}


In [62]:
response = requests.get("http://localhost:8000/intents")
print(response.json())

{'intents': ['cancel_order', 'change_order', 'change_shipping_address', 'check_cancellation_fee', 'check_invoice', 'check_payment_methods', 'check_refund_policy', 'complaint', 'contact_customer_service', 'contact_human_agent', 'create_account', 'delete_account', 'delivery_options', 'delivery_period', 'edit_account', 'get_invoice', 'get_refund', 'newsletter_subscription', 'payment_issue', 'place_order', 'recover_password', 'registration_problems', 'review', 'set_up_shipping_address', 'switch_account', 'track_order', 'track_refund']}


In [64]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class Query(BaseModel):
    text: str

@app.middleware("http")
async def log_requests(request: Request, call_next):
    start_time = datetime.now()
    
    request_body = await request.body()
    try:
        request_body = json.loads(request_body.decode())
    except:
        request_body = {}
    
    logger.info(f"Request: {request.method} {request.url} - Body: {request_body}")
    
    response = await call_next(request)
    
    process_time = (datetime.now() - start_time).total_seconds() * 1000
    logger.info(f"Response: {response.status_code} - Process Time: {process_time:.2f}ms")
    
    return response

In [71]:
@app.post("/predict")
async def predict_intent(query: Query) -> Dict[str, Any]:
    """Endpoint to predict intent from user text"""
    try:
        if not query.text or not isinstance(query.text, str):
            raise HTTPException(status_code=400, detail="Invalid input text")
        
        text = clean_text(query.text)
        inputs = eval_tokenizer(text, return_tensors="pt", padding="max_length", truncation=True)
        
        with torch.no_grad():
            outputs = eval_model(**inputs)
        
        pred_id = torch.argmax(outputs.logits, dim=1).item()
        intent = list(intent_labels.keys())[list(intent_labels.values()).index(pred_id)]
        confidence = torch.softmax(outputs.logits, dim=1)[0][pred_id].item()
        
        response = {
            "status": "success",
            "data": {
                "text": query.text,
                "intent": intent,
                "confidence": confidence,
                "timestamp": datetime.now().isoformat()
            }
        }
        
        return JSONResponse(content=response, status_code=200)
        
    except Exception as e:
        logger.error(f"Prediction error: {str(e)}", exc_info=True)
        error_response = {
            "status": "error",
            "message": "Failed to process request",
            "detail": str(e),
            "timestamp": datetime.now().isoformat()
        }
        return JSONResponse(content=error_response, status_code=500)

@app.get("/intents")
async def get_intents() -> Dict[str, Any]:
    """Endpoint to list all available intents"""
    try:
        response = {
            "status": "success",
            "data": {
                "intents": list(intent_labels.keys()),
                "count": len(intent_labels),
                "timestamp": datetime.now().isoformat()
            }
        }
        return JSONResponse(content=response, status_code=200)
    except Exception as e:
        logger.error(f"Intents list error: {str(e)}", exc_info=True)
        error_response = {
            "status": "error",
            "message": "Failed to retrieve intents",
            "timestamp": datetime.now().isoformat()
        }
        return JSONResponse(content=error_response, status_code=500)

In [72]:
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

time.sleep(2)

print("""
✅ Backend Server Running!
http://localhost:8000/docs - Swagger UI
http://localhost:8000/redoc - ReDoc UI
""")

INFO:     Started server process [4024]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 8000): only one usage of each socket address (protocol/network address/port) is normally permitted
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.



✅ Backend Server Running!
http://localhost:8000/docs - Swagger UI
http://localhost:8000/redoc - ReDoc UI



In [73]:
test_text = "How do I reset my password?"
response = requests.post("http://localhost:8000/predict", json={"text": test_text})
print("Prediction Test:")
print(json.dumps(response.json(), indent=2))

response = requests.get("http://localhost:8000/intents")
print("\nIntents Test:")
print(json.dumps(response.json(), indent=2))

Prediction Test:
{
  "text": "How do I reset my password?",
  "intent": "recover_password",
  "confidence": 0.9861235618591309
}

Intents Test:
{
  "intents": [
    "cancel_order",
    "change_order",
    "change_shipping_address",
    "check_cancellation_fee",
    "check_invoice",
    "check_payment_methods",
    "check_refund_policy",
    "complaint",
    "contact_customer_service",
    "contact_human_agent",
    "create_account",
    "delete_account",
    "delivery_options",
    "delivery_period",
    "edit_account",
    "get_invoice",
    "get_refund",
    "newsletter_subscription",
    "payment_issue",
    "place_order",
    "recover_password",
    "registration_problems",
    "review",
    "set_up_shipping_address",
    "switch_account",
    "track_order",
    "track_refund"
  ]
}


In [3]:
from transformers import DistilBertForSequenceClassification

model_path = r"C:\Users\hafiz\Documents\Gomycode\Test chatbot\chatbot_model"
model = DistilBertForSequenceClassification.from_pretrained(model_path)